In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
%matplotlib inline

from datetime import timedelta

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
                                                            
import xgboost as xgb
from sklearn import preprocessing
from sklearn.feature_selection import RFE, SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
# df_events información de eventos realizado en la plataforma para un conjunto de usuarios hasta el 31/05/2018
df_events = pd.read_csv('../../data/events_up_to_01062018.csv',dtype=object)
# df_train archivo labels_training_set.csv indica para un subconjunto de los usuarios incluidos en el set de 
# eventos events_up_to_01062018.csv si los mismos realizaron una conversión (columna label = 1) o no (columna 
# label = 0) desde el 01/06/2018 hasta el 15/06/2018
df_labels = pd.read_csv('../../data/labels_training_set.csv',dtype=object)
# Personas a predecir conversion.
df_person = pd.read_csv('../../data/trocafone_kaggle_test.csv')

MemoryError: 

In [3]:
df_labels['label'] = df_labels['label'].astype(int)
df_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414 entries, 0 to 19413
Data columns (total 2 columns):
person    19414 non-null object
label     19414 non-null int64
dtypes: int64(1), object(1)
memory usage: 303.4+ KB


In [11]:
def stratifiedCVConKBestChi2(data, K):
    
    df_train = pd.merge(data,df_labels,how = 'inner', on ='person')
    df_train.drop(columns = ['person'],inplace = True)
    
    X = df_train.drop(columns = ['label'])
    y = df_train['label']
    
    model = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)
    
    stratifiedKFold = StratifiedKFold(n_splits = 10, random_state = 7)
    
    selector = SelectKBest(chi2, k = K)
    selector.fit(X, y)
    X_new = selector.transform(X)
    
    nombre_columnas = list(X.columns[selector.get_support(indices=True)])
    
    X_new = pd.DataFrame(X_new)
    X_new.columns = nombre_columnas
    
    model.fit(X_new, y)
    
    df_kaggle_con_personas = pd.merge(df_person, data, how = 'inner', on = 'person')
    df_kaggle_sin_personas = df_kaggle_con_personas[list(X.columns[selector.get_support(indices=True)])]
    preds = model.predict(df_kaggle_sin_personas)
    predicciones = df_kaggle_con_personas[['person']]
    predicciones['label'] = preds
    
    predicciones.to_csv('ultima_prueba.csv', encoding='utf-8', index=False)
    
    results = cross_val_score(model, X_new, y, cv = stratifiedKFold, scoring = 'roc_auc')   
    
    print("ROC AUC: ",results.mean())
    print(list(X.columns[selector.get_support(indices=True)]))

In [5]:
df_seba2 = pd.read_csv('features_seba.csv')

In [6]:
df_featuresEventos = pd.read_csv('Santi_FeaturesConEventos.csv')
df_featuresEventos.drop(columns = ['Unnamed: 0'],inplace = True)

In [7]:
df_train = pd.merge(df_seba2,df_featuresEventos, on = 'person', how = 'inner')

In [8]:
df_train.shape

(38829, 194)

In [12]:
stratifiedCVConKBestChi2(df_train,139)

/home/santi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ROC AUC:  0.867080615221008
['ad campaign hit mes 5', 'brand listing mes 5', 'checkout mes 5', 'conversion mes 5', 'generic listing mes 5', 'search engine hit mes 5', 'searched products mes 5', 'staticpage mes 5', 'viewed product mes 5', 'visited site mes 5', 'dias_hasta_ultimo', 'ad campaign hit mes 4', 'brand listing mes 4', 'checkout mes 4', 'conversion mes 4', 'generic listing mes 4', 'lead mes 4', 'search engine hit mes 4', 'searched products mes 4', 'staticpage mes 4', 'viewed product mes 4', 'visited site mes 4', 'distan_dias', 'nuevo_mes_5', 'cant_dias_dist', 'modelos_dist', 'promedio_por_dia_x', 'promedio_por_mes', 'mes_primer_entrada', 'ad campaign hit', 'brand listing', 'checkout', 'conversion', 'generic listing', 'lead', 'search engine hit', 'searched products', 'staticpage', 'viewed product', 'visited site', 'vis_iPhone 6S Plus', 'vis_iPhone 7 Plus', 'vis_iPhone 6 Plus', 'vis_iPhone 6S', 'vis_Samsung Galaxy S7 Edge', 'vis_Samsung Galaxy S6 Flat', 'vis_iPhone SE', 'vis_Sams